In [ ]:
import os
from pathlib import Path
import sys

from dotenv import load_dotenv
import pandas as pd


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.utils.google_services import set_up_google_connection

In [ ]:
credentials_path = Path('../credentials/cool-plasma-452619-v4-feb20b70d461.json')

In [ ]:
client, service = set_up_google_connection(credentials_path)

load_dotenv()
SPREADSHEET_ID = os.getenv('SPREADSHEET_ID_2025')

In [ ]:
spreadsheet = client.open_by_key(SPREADSHEET_ID)

sheet_incomes = spreadsheet.worksheet('Einnahmen')
sheet_expenses = spreadsheet.worksheet('Ausgaben')

df_expenses = pd.DataFrame(sheet_expenses.get_all_values())
df_incomes = pd.DataFrame(sheet_incomes.get_all_values())

In [ ]:
df_expenses.columns = df_expenses.iloc[0]
df_expenses = df_expenses[1:].reset_index(drop=True)

df_incomes.columns = df_incomes.iloc[0]
df_incomes = df_incomes[1:].reset_index(drop=True)

gsheets = {'Expense': df_expenses, 'Income': df_incomes}

In [ ]:
gsheets['Income'].head()

In [ ]:
month_cols = [f'{i:02}' for i in range(1, 13)]

# Alles außer Ziffern und Komma/Punkt rauswerfen, dann zu Zahl
gsheets['Income'][month_cols] = (
    gsheets['Income'][month_cols]
    .replace(r'[^0-9,.-]', '', regex=True)  # Währungen, Leerzeichen, etc. entfernen
    .replace(',', '.', regex=True)  # Kommas durch Punkt ersetzen (falls Kommazahlen)
    .apply(pd.to_numeric, errors='coerce')  # endlich in float/int umwandeln
)

# Alles außer Ziffern und Komma/Punkt rauswerfen, dann zu Zahl
gsheets['Expense'][month_cols] = (
    gsheets['Expense'][month_cols]
    .replace(r'[^0-9,.-]', '', regex=True)  # Währungen, Leerzeichen, etc. entfernen
    .replace(',', '.', regex=True)  # Kommas durch Punkt ersetzen (falls Kommazahlen)
    .apply(pd.to_numeric, errors='coerce')  # endlich in float/int umwandeln
)

In [ ]:
gsheets['Income'].head()

In [ ]:
gsheets['Income'].dtypes

In [ ]:
gsheets['Income']['Kategorie'].unique()

# Questions to answer (Each Income and Expense)

- What is the sum of each month? -> Bar Chart
- What is the sum of each category? -> Pie chart

In [ ]:
import matplotlib.pyplot as plt

## What is the sum of each month?

In [ ]:
# Income
income_per_month = gsheets['Income'][month_cols].sum()
df_income_per_month = pd.DataFrame([income_per_month], index=['Income'])

# Expense
expense_per_month = gsheets['Expense'][month_cols].sum()
df_expense_per_month = pd.DataFrame([expense_per_month], index=['Expense'])

df_all_sums = pd.concat([df_income_per_month, df_expense_per_month])
df_all_sums

In [ ]:
df_all_sums.T.plot(kind='line', marker='o', figsize=(10, 6))
plt.title('Income vs. Expense per Month')
plt.xlabel('Month')
plt.ylabel('Amount (€)')
plt.grid(True)
plt.show()

In [ ]:
df_all_sums.T.plot(kind='bar', figsize=(12, 6))
plt.title('Monthly Income vs. Expense')
plt.xlabel('Month')
plt.ylabel('Amount (€)')
plt.legend(title='')
plt.tight_layout()
plt.show()

In [ ]:
(df_all_sums.loc['Income'] - df_all_sums.loc['Expense']).plot(
    kind='bar', color='green', figsize=(10, 6)
)
plt.axhline(0, color='red', linestyle='--')
plt.title('Net Income (Profit/Loss) per Month')
plt.xlabel('Month')
plt.ylabel('Net Income (€)')
plt.tight_layout()
plt.show()

In [ ]:
totals = df_all_sums.sum(axis=1)
totals['Net Income'] = totals['Income'] - totals['Expense']

print(totals)

totals.plot(kind='bar', color=['blue', 'red', 'green'], figsize=(8, 5))
plt.title('Yearly Totals: Income, Expense, Net Income')
plt.ylabel('Amount (€)')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

## What is the sum of each category?

### Income

In [ ]:
income_per_cat = gsheets['Income'].groupby('Kategorie')[month_cols].sum()
income_per_cat['Total'] = income_per_cat.sum(axis=1)
df_income = income_per_cat.reset_index()[['Kategorie', 'Total']]

In [ ]:
# PieChart zeichnen
plt.figure(figsize=(8, 8))
plt.pie(
    df_income['Total'],
    labels=df_income['Kategorie'],
    autopct='%1.1f%%',
    startangle=90,
    counterclock=False,
)
plt.title('Anteil Gesamt pro Kategorie')
plt.tight_layout()
plt.show()

### Expense

In [ ]:
expense_per_cat = gsheets['Expense'].groupby('Kategorie')[month_cols].sum()
expense_per_cat['Total'] = expense_per_cat.sum(axis=1)
df_expense = expense_per_cat.reset_index()[['Kategorie', 'Total']]

In [ ]:
df_expense.sort_values(by='Total', ascending=False)

In [ ]:
plt.figure(figsize=(8, 8))
plt.pie(
    df_expense['Total'],
    labels=df_expense['Kategorie'],
    autopct='%1.1f%%',
    startangle=90,
    counterclock=False,
)
plt.title('Anteil Gesamt pro Kategorie')
plt.tight_layout()
plt.show()